<a href="https://colab.research.google.com/github/Pradeep2535/Colab-Notebooks-for-GenAI/blob/main/Hybrid_Search_Reranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
#Sample documents

documents = [



"This is a list which containig sample documents.",

"Keywords are important for keyword-based search.",

"Document analysis involves extracting keywords.",

"Keyword-based search relies on sparse embeddings."
]

In [ ]:
import re

def preprocess_text(text):

  text =text.lower()
  text =re.sub(r' [^\w\s]', '', text)

  return text

In [ ]:
preprocess_documents = [preprocess_text(doc) for doc in documents]

In [ ]:
preprocess_documents

In [ ]:
preprocessed_query = preprocess_text("Keyword-based search")

In [ ]:
preprocessed_query

In [ ]:
vector = TfidfVectorizer()

In [ ]:
X = vector.fit_transform(preprocess_documents)

In [ ]:
X.toarray()

In [ ]:
query_embedding=vector.transform([preprocessed_query])

In [ ]:
query_embedding.toarray()

In [ ]:
similarities = cosine_similarity(X, query_embedding)

In [ ]:
similarities

In [ ]:
ranked_documents = [documents[i] for i in ranked_indices]

In [ ]:
ranked_indices=np.argsort(similarities, axis=0)[::-1].flatten()

In [ ]:
ranked_indices

In [ ]:
for i,doc in enumerate(ranked_documents):
  print(f"Rank {i+1}: {doc}")

In [ ]:
query="Keyword-based search"

In [ ]:
document_embeddings = np.array([
[0.634, 0.234, 0.867, 0.042, 0.249],
[0.123, 0.456, 0.789, 0.321, 0.654],
[0.987, 0.654, 0.321, 0.123, 0.456]])

In [ ]:
query_embedding = np.array([[0.789, 0.321,0.654,0.987,0.123]])

In [ ]:
similarities = cosine_similarity(document_embeddings, query_embedding)

In [ ]:
ranked_indices=np.argsort(similarities, axis=0)[::-1].flatten()

In [ ]:
ranked_indices=np.argsort(similarities, axis=0)[::-1].flatten()

In [ ]:
ranked_indices

In [ ]:
for i,idx in enumerate(ranked_indices):
  print(f"Rank {i+1}: Document {idx+1}")

In [ ]:
doc_path = "/content/2005.11401v4.pdf"

In [ ]:
!pip install pypdf langchain_community

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
loader=PyPDFLoader(doc_path)

In [ ]:
docs=loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=30)

In [ ]:
chunks = splitter.split_documents(docs)

In [ ]:
chunks

In [ ]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

In [ ]:
from google.colab import userdata

HF_TOKEN=userdata.get("HF_TOKEN")

In [ ]:
embeddings = HuggingFaceInferenceAPIEmbeddings(api_key=HF_TOKEN,model_name="BAAI/bge-base-en-v1.5")

In [ ]:
!pip install chromadb

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
vectorstore=Chroma.from_documents(chunks,embeddings)

In [ ]:
vectorstore_retriever=vectorstore.as_retriever(search_kwargs={"k":3})

In [ ]:
!pip install rank_bm25

In [ ]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

In [ ]:
keyword_retreiver = BM25Retriever.from_documents(chunks)

In [ ]:
retreiver = EnsembleRetriever(retrievers=[vectorstore_retriever,keyword_retreiver],weights=[0.5,0.5])

In [ ]:
model_name="HuggingFaceH4/zephyr-7b-beta"

In [ ]:
!pip install -U bitsandbytes

In [ ]:
!pip install accelerate

In [ ]:
import torch
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline)
from langchain import HuggingFacePipeline

In [ ]:
def load_quantized_model(model_name: str):



  bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
  )

  model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
  )

  return model

In [ ]:
def initialize_tokenizer(model_name: str):
  tokenizer =AutoTokenizer.from_pretrained(model_name, return_token_type_ids=False)

  tokenizer.bos_token_id = 1

  return tokenizer

In [ ]:
tokenizer=initialize_tokenizer(model_name)

In [ ]:
model   =load_quantized_model(model_name)

In [ ]:
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,use_cache=True,device_map="auto",max_length=2048,do_sample=True,top_k=5,num_return_sequences=1,eos_token_id = tokenizer.eos_token_id, pad_token_id = tokenizer.pad_token_id
)


In [ ]:
llm=HuggingFacePipeline(pipeline=pipeline)

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
normal_chain = RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",retriever=vectorstore_retriever)

In [ ]:
hybrid_chain = RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",retriever=retreiver)

In [ ]:
response1=normal_chain.invoke("What is RAG token model?")

In [ ]:
print(response1["result"])

In [ ]:

response2=hybrid_chain.invoke("What is RAG token model?")

In [ ]:
response2

In [ ]:
print(response2["result"])

#Reranking

In [ ]:
!pip install cohere

In [ ]:
COHERE_API = userdata.get("COHERE_API")

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

In [ ]:
compressor = CohereRerank(cohere_api_key=COHERE_API)

In [ ]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retreiver
    )

In [ ]:
compressed_docs = compression_retriever.get_relevant_documents("What is RAG token model?")

In [ ]:
hybrid_chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=compression_retriever
)

In [ ]:
response = hybrid_chain.invoke("What is RAG token model?")

In [ ]:
response

In [ ]:
print(response["result"])